In [1]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(), '..','..','..'))
import config, utils

# Create a Spark session
spark = utils.create_spark_session("playerStats", {
    'spark.executor.memory': '4g',
    'spark.executor.cores': '6',
})

fielding_data = utils.load_data(spark,config.PROCESSED_DATA_DIR, 'fielding_data.csv')
bowling_data = utils.load_data(spark,config.PROCESSED_DATA_DIR, 'bowling_data.csv')
batting_data = utils.load_data(spark,config.PROCESSED_DATA_DIR, 'batting_data.csv')

fielding_data.show(5)

/home/ravikumar/miniconda3/envs/t20i/lib/python3.12/site-packages/airflow/configuration.py:859 FutureWarning: section/key [core/sql_alchemy_conn] has been deprecated, you should use[database/sql_alchemy_conn] instead. Please update your `conf.get*` call to use the new name

[2024-11-15T15:12:07.985+0530] {utils.py:12} INFO - Creating Spark session.


your 131072x1 screen size is bogus. expect trouble
24/11/15 15:12:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/15 15:12:15 WARN Utils: spark.executor.instances less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, please update your configs.


[2024-11-15T15:12:16.087+0530] {utils.py:30} INFO - Spark session created successfully.
[2024-11-15T15:12:16.089+0530] {utils.py:39} INFO - Loading data from fielding_data.csv.


[2024-11-15T15:12:24.316+0530] {utils.py:39} INFO - Loading data from bowling_data.csv.


[2024-11-15T15:12:24.817+0530] {utils.py:39} INFO - Loading data from batting_data.csv.
+----------+--------+-------+--------------+---------------+--------------+-------------+-------------+--------------+
|    Player| Country| Season|Cumulative Mat|Cumulative Inns|Cumulative Dis|Cumulative Ct|Cumulative St|Cumulative D/I|
+----------+--------+-------+--------------+---------------+--------------+-------------+-------------+--------------+
|A Ahmadhel|Bulgaria|2019/20|             0|            0.0|           0.0|          0.0|          0.0|           0.0|
|A Ahmadhel|Bulgaria|   2020|             3|            3.0|           0.0|          0.0|          0.0|           0.0|
|A Ahmadhel|Bulgaria|2020/21|             4|            4.0|           0.0|          0.0|          0.0|           0.0|
|A Ahmadhel|Bulgaria|   2021|             6|            6.0|           0.0|          0.0|          0.0|           0.0|
|A Ahmadhel|Bulgaria|   2023|             9|            9.0|           0.0|    

In [2]:
# Check for nulls in the data
def check_nulls(df):
    nulls = 0
    for col in df.columns:
        nulls+= df.filter(df[col].isNull()).count()
    print("Total nulls in the dataframe: ", nulls)

check_nulls(batting_data)
check_nulls(bowling_data)
check_nulls(fielding_data)

Total nulls in the dataframe:  0
Total nulls in the dataframe:  0
Total nulls in the dataframe:  0


In [3]:
print(batting_data.columns,batting_data.count())
print(bowling_data.columns,bowling_data.count())
print(fielding_data.columns,fielding_data.count())

['Player', 'Country', 'Season', 'Cum Mat Total', 'Cum Inns Total', 'Cum Runs Total', 'Cum Batting Ave', 'Cum SR'] 14142
['Player', 'Country', 'Season', 'Cumulative Mat', 'Cumulative Inns', 'Cumulative Overs', 'Cumulative Runs', 'Cumulative Wkts', 'Cumulative Econ'] 14142
['Player', 'Country', 'Season', 'Cumulative Mat', 'Cumulative Inns', 'Cumulative Dis', 'Cumulative Ct', 'Cumulative St', 'Cumulative D/I'] 14142


In [5]:
playerdata = batting_data.join(bowling_data, on=['Player',"Country","Season"], how='inner').join(fielding_data, on=['Player',"Country","Season"], how='inner')\
    .drop('Cumulative Mat','Cumulative Inns')
playerdata = playerdata.dropDuplicates([ 'Player', 'Country', 'Season'])
print(playerdata.count())
playerdata.show(5)

14142


24/11/15 15:13:02 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------+----------+-------+-------------+--------------+--------------+---------------+------+------------------+---------------+---------------+---------------+--------------+-------------+-------------+--------------+
|        Player|   Country| Season|Cum Mat Total|Cum Inns Total|Cum Runs Total|Cum Batting Ave|Cum SR|  Cumulative Overs|Cumulative Runs|Cumulative Wkts|Cumulative Econ|Cumulative Dis|Cumulative Ct|Cumulative St|Cumulative D/I|
+--------------+----------+-------+-------------+--------------+--------------+---------------+------+------------------+---------------+---------------+---------------+--------------+-------------+-------------+--------------+
|     AU Rashid|   England|   2023|           95|            30|            92|           4.62|  78.5|331.20000076293945|         2454.0|           95.0|           7.55|          28.0|         28.0|          0.0|           0.3|
|    Aizaz Khan| Hong Kong|2022/23|           49|            42|           429|         

In [6]:
playerdata.sort(["Player","Country"]).show()

+--------------+--------------+-------+-------------+--------------+--------------+---------------+------+------------------+---------------+---------------+---------------+--------------+-------------+-------------+--------------+
|        Player|       Country| Season|Cum Mat Total|Cum Inns Total|Cum Runs Total|Cum Batting Ave|Cum SR|  Cumulative Overs|Cumulative Runs|Cumulative Wkts|Cumulative Econ|Cumulative Dis|Cumulative Ct|Cumulative St|Cumulative D/I|
+--------------+--------------+-------+-------------+--------------+--------------+---------------+------+------------------+---------------+---------------+---------------+--------------+-------------+-------------+--------------+
|    A Ahmadhel|      Bulgaria|2020/21|            4|             3|            24|            8.0| 100.0|              12.0|           97.0|            4.0|           8.38|           0.0|          0.0|          0.0|           0.0|
|    A Ahmadhel|      Bulgaria|   2024|           11|             7|    

In [7]:
utils.spark_save_data(playerdata,config.PROCESSED_DATA_DIR, 'player_stats.csv')
spark.stop()

[2024-11-15T15:13:35.326+0530] {utils.py:57} INFO - Successfully wrote data to /usr/ravi/t20/data/2_processedData/player_stats.csv
